In [33]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm.notebook import tqdm
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.integrate import ode
import seaborn as sns


# Original Parameter values


## Initial values

In [34]:
N_h= 1.29e9 + 250+89+2e5
N_d = 3.5e7 +2e5+1e5+2e5

E_d = 2e5/N_d
I_d = 1e5/N_d
Q_d = 0
V_d = 2e5/N_d

E_h = 250/N_h
I_h = 89/N_h
V_h = 2e5/N_h

S_d = 3.5e7/N_d
S_h = 1.29e9/N_h

In [35]:
λ_h = 1.54e7
λ_d = 3e6

μ_h = 0.0066
μ_e = 0.3 ######
μ_d = 0.08
μ_rd= 1
μ_rh= 1

r_1 = 365/45 #####
δ = 0.01 #####
σ_d= 0.12 ######

β_dd= 1.58e-7 * N_d
β_dh= 2.29e-12 * N_h

ψ_d=0.09
ψ_h=0.54

γ_d=6
γ_h=6

χ_d=0.4
χ_h=0.4
 
α_d=1
α_h=1

## ODE

In [36]:
def f(t, x):
    
   S_d, E_d, I_d, Q_d, V_d,    S_h,E_h,I_h,V_h = x
   
  

   dS_d = λ_d / N_d -(μ_d + ψ_d)*S_d -(β_dd*I_d*S_d)/(1-Q_d)  + γ_d*(1-χ_d)*E_d +α_d*V_d + r_1*Q_d
   dE_d = -(μ_e + γ_d + σ_d + μ_d + ψ_d)*E_d + (β_dd*I_d*(S_d+V_d*δ))/(1-Q_d)
   dI_d = γ_d*χ_d*E_d -(μ_d + μ_rd)*I_d
   dQ_d = σ_d*E_d -(r_1 + μ_d + μ_e +  μ_rd)*Q_d 
   dV_d = -(μ_d + α_d)*V_d -(β_dd*I_d*V_d*δ)/(1-Q_d) + ψ_d*(S_d  + E_d)
    
   dS_h = λ_h/N_h -(μ_h +ψ_h)*S_h -(β_dh*I_d*S_h)*N_d/N_h  + γ_h*(1-χ_h)*E_h +α_h*V_h 
   dE_h = (β_dh*I_d*(S_h+V_h*δ)*N_d/N_h) - (γ_h + μ_h + ψ_h)*E_h
   dI_h = γ_h*χ_h*E_h -(μ_h + μ_rh)*I_h
   dV_h = -(μ_h + α_h)*V_h -(β_dh*I_d*V_h*δ)*N_d/N_h + ψ_h*(S_h + E_h) 
    
   return [dS_d, dE_d, dI_d, dQ_d, dV_d, dS_h, dE_h, dI_h, dV_h]

In [37]:
params = np.array([λ_h,λ_d,μ_h,μ_e,μ_d,μ_rd,μ_rh,r_1,δ,σ_d,β_dd,β_dh,
                    ψ_d,ψ_h,γ_d,γ_h,χ_d,χ_h,α_d,α_h])
string_params = ['λ_h','λ_d','μ_h','μ_e','μ_d','μ_rd','μ_rh','r_1','δ','σ_d','β_dd','β_dh',
                    'ψ_d','ψ_h','γ_d','γ_h','χ_d','χ_h','α_d','α_h']

In [38]:
x0 = [S_d, E_d, I_d, Q_d, V_d,    #dogs
      S_h, E_h, I_h, V_h]  #humans

t_span=(0,50)

In [39]:
original_ivp_version = solve_ivp(f, t_span=(0,50), y0=x0, method="BDF", rtol=1e-8, t_eval=np.linspace(0,50,t_span[1]+1))#, max_step=1e-4)
#convert into lists
original = np.zeros((len(range(t_span[1]+1)),9))
for j in range(0,9):
    original[:,j] = original_ivp_version.y[j]

## Repeatedly Solve ODE with perturbed Parameters

In [40]:
def perturb_params(n, original_params, scale):
    
    solutions = []

    for i in tqdm(range(n), leave = False):
        perturbed_params = np.random.uniform( (1-scale)*original_params, (1+scale)*original_params)
        
        λ_h,λ_d,μ_h,μ_e,μ_d,μ_rd,μ_rh,r_1,δ,σ_d,β_dd,β_dh, ψ_d,ψ_h,γ_d,γ_h,χ_d,χ_h,α_d,α_h = perturbed_params

        
        x0 = [S_d, E_d, I_d, Q_d, V_d,    #dogs
              S_h, E_h, I_h, V_h]

        #define function again to get new params
        def f(t, x):
    
            S_d, E_d, I_d, Q_d, V_d,    S_h,E_h,I_h,V_h = x
   
  

            dS_d = λ_d / N_d -(μ_d + ψ_d)*S_d -(β_dd*I_d*S_d)/(1-Q_d)  + γ_d*(1-χ_d)*E_d +α_d*V_d + r_1*Q_d
            dE_d = -(μ_e + γ_d + σ_d + μ_d + ψ_d)*E_d + (β_dd*I_d*(S_d+V_d*δ))/(1-Q_d)
            dI_d = γ_d*χ_d*E_d -(μ_d + μ_rd)*I_d
            dQ_d = σ_d*E_d -(r_1 + μ_d + μ_e +  μ_rd)*Q_d 
            dV_d = -(μ_d + α_d)*V_d -(β_dd*I_d*V_d*δ)/(1-Q_d) + ψ_d*(S_d  + E_d)
    
            dS_h = λ_h/N_h -(μ_h +ψ_h)*S_h -(β_dh*I_d*S_h)*N_d/N_h  + γ_h*(1-χ_h)*E_h +α_h*V_h 
            dE_h = (β_dh*I_d*(S_h+V_h*δ)*N_d/N_h) - (γ_h + μ_h + ψ_h)*E_h
            dI_h = γ_h*χ_h*E_h -(μ_h + μ_rh)*I_h
            dV_h = -(μ_h + α_h)*V_h -(β_dh*I_d*V_h*δ)*N_d/N_h + ψ_h*(S_h + E_h) 

            return [dS_d, dE_d, dI_d, dQ_d, dV_d, dS_h, dE_h, dI_h, dV_h]
        solutions.append(solve_ivp(f, t_span=(0,50), y0=x0, method="BDF", rtol=1e-8,t_eval=np.linspace(t_span[0],t_span[1],t_span[1]+1)))#, max_step=1e-4))
        #print(solutions[0].y)
        
    #convert into the right shapes
    A_solutions = np.zeros((n,len(range(t_span[1]+1)),9))
    for i in range(0,n):
        for j in range(0,9):
            A_solutions[i,:,j] = solutions[i].y[j]
        
    return A_solutions

## Functions for finding the minimum and maximum deviations

In [41]:
def get_min_max_2(solutions):
    sol_arr = np.array(solutions)
    #get min/max solutions
    S_max = np.amax(sol_arr, axis=0)
    S_min = np.amin(sol_arr, axis=0)
    #get index of min/max solutions, index is taken from the first entry, if multiple are available
    S_max_index = np.argmax(sol_arr, axis=0)
    S_min_index = np.argmin(sol_arr, axis=0)

    return [S_max, S_min, S_max_index, S_min_index]

In [42]:
def find_max_min_diviation_param(sol_max, sol_min, original_curves, index_max, index_min, changed_params):
    save_max_param_per_compartment = []
    save_min_param_per_compartment = []

    k, n = sol_max.shape
    print(original_curves.shape)
    for i in range(n):
        ind_max = np.argmax(sol_max[:,i]-original_curves[:,i])
        ind_min = np.argmin(sol_min[:,i]-original_curves[:,i]) #takes the most negative one
        save_max_param_per_compartment.append(changed_params[index_max[ind_max,i]])
        save_min_param_per_compartment.append(changed_params[index_min[ind_min,i]])
    return [save_max_param_per_compartment, save_min_param_per_compartment]

In [43]:
def perturb_single_param_2(n, original_params, scale, index):

    mesh = np.linspace(original_params[index]*(1-scale), original_params[index]*(1+scale), n)
    solutions = []
    parameter_diviations = []
    ts = np.linspace(0,50, t_span[1]+1)

    for i in tqdm(range(n)):
        perturbed_params = np.copy(original_params)
        perturbed_params[index] = mesh[i]
        parameter_diviations.append(mesh[i])
        λ_h,λ_d,μ_h,μ_e,μ_d,μ_rd,μ_rh,r_1,δ,σ_d,β_dd,β_dh, ψ_d,ψ_h,γ_d,γ_h,χ_d,χ_h,α_d,α_h = perturbed_params

        
        x0 = [S_d, E_d, I_d, Q_d, V_d,    #dogs
              S_h, E_h, I_h, V_h]

        #define function again to get new params
        def f(t, x):
    
            S_d, E_d, I_d, Q_d, V_d,    S_h,E_h,I_h,V_h = x
   
  

            dS_d = λ_d / N_d -(μ_d + ψ_d)*S_d -(β_dd*I_d*S_d)/(1-Q_d)  + γ_d*(1-χ_d)*E_d +α_d*V_d + r_1*Q_d
            dE_d = -(μ_e + γ_d + σ_d + μ_d + ψ_d)*E_d + (β_dd*I_d*(S_d+V_d*δ))/(1-Q_d)
            dI_d = γ_d*χ_d*E_d -(μ_d + μ_rd)*I_d
            dQ_d = σ_d*E_d -(r_1 + μ_d + μ_e +  μ_rd)*Q_d 
            dV_d = -(μ_d + α_d)*V_d -(β_dd*I_d*V_d*δ)/(1-Q_d) + ψ_d*(S_d  + E_d)
    
            dS_h = λ_h/N_h -(μ_h +ψ_h)*S_h -(β_dh*I_d*S_h)*N_d/N_h  + γ_h*(1-χ_h)*E_h +α_h*V_h 
            dE_h = (β_dh*I_d*(S_h+V_h*δ)*N_d/N_h) - (γ_h + μ_h + ψ_h)*E_h
            dI_h = γ_h*χ_h*E_h -(μ_h + μ_rh)*I_h
            dV_h = -(μ_h + α_h)*V_h -(β_dh*I_d*V_h*δ)*N_d/N_h + ψ_h*(S_h + E_h) 

            return [dS_d, dE_d, dI_d, dQ_d, dV_d, dS_h, dE_h, dI_h, dV_h]
        solutions.append(solve_ivp(f, t_span=(0,50), y0=x0, method="BDF", rtol=1e-8,t_eval=np.linspace(t_span[0],t_span[1],t_span[1]+1)))#, max_step=1e-4))

    #convert into the right shapes
    A_solutions = np.zeros((n,len(range(t_span[1]+1)),9))
    for i in range(0,n):
        for j in range(0,9):
            A_solutions[i,:,j] = solutions[i].y[j]

    return A_solutions, parameter_diviations

# Functions for an automatic plot

In [44]:
def plot_compartments_2(original_curves, perturbed_params,  single_solution, string_params, index, file_specific_range = False):
    compartment_names = ['S_d', 'E_d', 'I_d', 'Q_d', 'V_d',    #humans
                         'S_h', 'E_h', 'I_h', 'V_h'] #x_6 = S_h
    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_solution)
    max_params_per_comp, min_params_per_comp = find_max_min_diviation_param(S_max, S_min, original, S_max_index, S_min_index, perturbed_params)


   
    figure, axis = plt.subplots(5, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(5):
        axis[i].plot(range(t_span[1]+1), S_max[:,i], color = 'r')
        axis[i].plot(range(t_span[1]+1), original_curves[:,i], color = 'g')
        axis[i].plot(range(t_span[1]+1), S_min[:,i], color = 'b')
        axis[i].legend(['max ' + string_params[index] +': ' + str(max_params_per_comp[i]),
                        'original ' + string_params[index]+': '+ str(params[index]) ,
                        'min ' + string_params[index] +': '+ str(min_params_per_comp[i])])
        axis[i].set_title(compartment_names[i], fontsize = 16)

    if file_specific_range == True:
        figure.savefig('perturb_specific_range/dog_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)
    else:
        figure.savefig('perturbed_compartment_plots/dog_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)

    plt.close(figure)
    plt.close('all')


    figure, axis = plt.subplots(4, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(5,9):
        axis[i-5].plot(range(t_span[1]+1), S_max[:,i], color = 'r')
        axis[i-5].plot(range(t_span[1]+1), original_curves[:,i], color = 'g')
        axis[i-5].plot(range(t_span[1]+1), S_min[:,i], color = 'b')
        axis[i-5].legend(['max ' + string_params[index] +': ' + str(max_params_per_comp[i]),
                        'original ' + string_params[index]+': '+ str(params[index]) ,
                        'min ' + string_params[index] +': '+ str(min_params_per_comp[i])])
        axis[i-5].set_title(compartment_names[i], fontsize = 16)

    if file_specific_range == True:
        figure.savefig('perturb_specific_range/human_plot_' + string_params[index] +
                       '_perturbed.jpg',bbox_inches = 'tight', dpi=500)
    else:
        figure.savefig('perturbed_compartment_plots/human_plot_' + string_params[index] +
                       '_perturbed.jpg', bbox_inches = 'tight', dpi=500)
    plt.close(figure)
    plt.close('all')

In [45]:
def plot_compartments_all(original_curves, single_solution, string_params, index, file_specific_range = False):
    compartment_names = ['S_d', 'E_d', 'I_d', 'Q_d', 'V_d',    #humans
                         'S_h', 'E_h', 'I_h', 'V_h'] # x_6=S_h

    S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_solution)

    
    figure, axis = plt.subplots(5, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(5):
        axis[i].plot(range(t_span[1]+1), S_max[:,i], color = 'r')
        axis[i].plot(range(t_span[1]+1), original_curves[:,i], color = 'g')
        axis[i].plot(range(t_span[1]+1), S_min[:,i], color = 'b')
        axis[i].legend(['max','original','min'])
        axis[i].set_title(compartment_names[i], fontsize = 16)

    figure.savefig('perturbed_compartment_plots/dog_plot_' + string_params[index] +
                   '_perturbed.jpg', bbox_inches='tight', dpi=500)
    plt.close(figure)
    plt.close('all')

   
    figure, axis = plt.subplots(4, 1)
    plt.rcParams["figure.figsize"] = (25/2,25/2)
    for i in range(5,9):
        axis[i-5].plot(range(t_span[1]+1), S_max[:,i], color = 'r')
        axis[i-5].plot(range(t_span[1]+1), original_curves[:,i], color = 'g')
        axis[i-5].plot(range(t_span[1]+1), S_min[:,i], color = 'b')
        axis[i-5].legend(['max','original','min'])
        axis[i-5].set_title(compartment_names[i], fontsize = 16)

    figure.savefig('perturbed_compartment_plots/human_plot_' + string_params[index] +
                   '_perturbed.jpg', bbox_inches='tight', dpi=500)
    plt.close(figure)
    plt.close('all')

# Functions for calculating and ordering the two uncertainty measures

In [46]:
def find_most_uncertain_parameter(list_of_matrices,list_of_parameters):
    n = len(list_of_matrices) #this corresponds to the number of parameters


    #Calculate uncertainty by adding up all differences of the min and max curves
    vector_rowSum = []
    sum_all_compartments = []
    for i in range(n):
        A = np.matrix(np.copy(list_of_matrices[i]))
        v = A.sum(axis=0)
        deviation_all_compartments = np.sum(v)
        vector_rowSum.append(deviation_all_compartments)
        sum_all_compartments.append(v)
    #we want a list of all parameters and their uncertainty value
    ordered_by_uncertainty = []
    #if the largest value is recorded, then the value is set to -1, to get the second biggest value.
    for i in range(len(vector_rowSum)):
        k = np.argmax(vector_rowSum)
        value, name = vector_rowSum[k]/50, list_of_parameters[k]
        ordered_by_uncertainty.append([name, value])
        #the second largest. All values in these matrices are positive, so we simply put it to -1
        vector_rowSum[k] = -1


    return ordered_by_uncertainty

## Plot all curves and calculate the sensitivity

### You need a folder with the name: "perturbed_compartment_plots" in your path

In [47]:
def plot_all_uncertainty_with_params(original_curves, params, string_params, n_perturb, perturb_scale):

    list_of_maxMinusmin = []
    for i in range(len(params)):
        single_perturbed_sol, parameter_perturbed = perturb_single_param_2(n_perturb, params, perturb_scale, i)
        S_max, S_min, S_max_index, S_min_index = get_min_max_2(single_perturbed_sol)
        list_of_maxMinusmin.append(S_max-S_min)
        plot_compartments_2(original_curves, parameter_perturbed, single_perturbed_sol, string_params, i)
        print('Done with parameter ' + string_params[i])

    #calculate the min/ max diviation quantitatively
    print('Ordered from largest to smallest influence overall')
    all_params_ordered_by_uncertainty = find_most_uncertain_parameter(list_of_maxMinusmin, string_params)
    df = pd.DataFrame(all_params_ordered_by_uncertainty, columns=['param_name', 'value'])
    df.to_csv('all_params_ordered_by_uncertainty.csv')
    print(df)


    #add plot: all params are perturbed
    all_perturbed = perturb_params(n_perturb, params, perturb_scale)
    plot_compartments_all(original_curves,  all_perturbed, ['all'],0)
    print('All parameters were perturbed. The computation is done.')

## Perform perturbation and uncertainty analysis

The calculations and plots are carried out automatically.

The plots are saved in "perturbed_compartment_plots". The results of the uncertainty analysis is stored in two files:

all_params_ordered_by_uncertainty.csv
all_params_ordered_by_uncertainty_influence_on_I.csv

In [49]:
plot_all_uncertainty_with_params(original, params, string_params,50000, 0.05)

  0%|          | 0/50000 [00:00<?, ?it/s]

(51, 9)
Done with parameter λ_h


  0%|          | 0/50000 [00:00<?, ?it/s]

(51, 9)
Done with parameter λ_d


  0%|          | 0/50000 [00:00<?, ?it/s]

...

(51, 9)
Done with parameter α_d


  0%|          | 0/50000 [00:00<?, ?it/s]

(51, 9)
Done with parameter α_h
Ordered from largest to smallest influence overall
   param_name         value
0         χ_d  6.731819e-02
1        β_dd  6.670692e-02
2        μ_rd  5.943129e-02
3         ψ_h  5.096888e-02
4         α_h  4.973136e-02
5         λ_h  2.731926e-02
6         λ_d  1.971618e-02
7         μ_h  1.638940e-02
8         μ_d  1.374598e-02
9         γ_d  8.441185e-03
10        ψ_d  7.483496e-03
11        α_d  6.308871e-03
12        μ_e  3.280544e-03
13        σ_d  1.214161e-03
14          δ  6.526619e-05
15        r_1  6.461373e-05
16       β_dh  1.951764e-06
17        χ_h  1.894409e-06
18        γ_h  1.715889e-07
19       μ_rh  7.296012e-08


  0%|          | 0/50000 [00:00<?, ?it/s]

All parameters were perturbed. The computation is done.
